# SkySat-snow pipeline

In [1]:
import os
import sys

# Define paths in directory
code_dir = '/Users/raineyaberle/Research/PhD/SnowDEMs/skysat-snow'
ref_dem_fn = '/Volumes/LaCie/raineyaberle/Research/PhD/Skysat-Stereo/study-sites/MCS/refdem/MCS_REFDEM_WGS84.tif'
pc_fn = '/Volumes/LaCie/raineyaberle/Research/PhD/Skysat-Stereo/study-sites/MCS/20240420/MCS_20240420_pc_merged.laz'
roads_fn = '/Volumes/LaCie/raineyaberle/Research/PhD/Skysat-Stereo/study-sites/MCS/roads/MCS_roads_line_merged_UTM11N.shp'
multispec_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/SkySat-Stereo/study-sites/MCS/20240420/MCS_20240420_4band_mosaic.tif'
asp_dir = '/Users/raineyaberle/Research/PhD/SnowDEMs/StereoPipeline-3.5.0-alpha-2024-10-05-x86_64-OSX/bin'
out_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/Skysat-Stereo/study-sites/MCS/20240420'
res = 2 # spatial resolution of outputs [m]
roads_buffer = 5 # buffer for roads geospatial file [m]

# Check that input files and directories exist
if not os.path.exists(ref_dem_fn):
    print('Reference DEM file not found, please correct ref_dem_fn before continuing.')
if not os.path.exists(pc_fn):
    print('Point cloud file not found, please correct pc_fn before continuing.')
if not os.path.exists(roads_fn):
    print('Roads geospatial file not found, please correct roads_fn before continuing.')
if not os.path.exists(multispec_dir):
    print('Multispectral images folder not found, please correct multispec_dir before continuing.')
if not os.path.exists(asp_dir):
    print('Path to ASP not found, please correct asp_dir before continuing.')
out_dir = os.path.join(out_dir, 'skysat_snow')
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
    print('Made directory for outputs:', out_dir)

# Make results directories
preprocess_dir = os.path.join(out_dir, 'preprocess')
masks_dir = os.path.join(out_dir, 'land_cover_masks')
deramp_dir = os.path.join(out_dir, 'deramp')
pc_align_dir = os.path.join(out_dir, 'pc_align')
pc_transform_dir = os.path.join(out_dir, 'pc_transform')
final_align_dir = os.path.join(out_dir, 'final_align')
for folder in [preprocess_dir, masks_dir, deramp_dir, pc_align_dir, pc_transform_dir, final_align_dir]:
    if not os.path.exists(folder):
        os.mkdir(folder)

# Add path to pipeline utilities
sys.path.append(os.path.join(code_dir, 'scripts'))
import pipeline_utils_pc as f

In [ ]:
##### 1. PREPROCESSING #####
# Merge and filter point clouds
pc_filtered_las_fn, pc_filtered_tif_fn = f.preprocess_point_clouds(pc_fn, ref_dem_fn, res, preprocess_dir)
# Mosaic 4-band SR imagery, preprocess roads and reference DEM
multispec_mosaic_fn, roads_adj_fn, ref_dem_adj_fn = f.preprocess_multispec_refdem_roads(multispec_dir, ref_dem_fn, roads_fn, roads_buffer, res, out_dir)


##### 2. CONSTRUCT LAND COVER MASKS #####
multispec_mosaic_fn, trees_mask_fn, snow_mask_fn, roads_mask_fn, ss_mask_fn = f.construct_land_cover_masks(multispec_mosaic_fn, 
                                                                                                           roads_adj_fn, 
                                                                                                           masks_dir, 
                                                                                                           roads_buffer,
                                                                                                           ndvi_threshold=0.2,
                                                                                                           ndsi_threshold=0.0)


##### 3. DERAMP THE DEM ########
pc_deramped_fn = f.deramp(ref_dem_fn, pc_filtered_tif_fn, pc_filtered_las_fn, deramp_dir, vmin=-10, vmax=10)


##### 4. ALIGN AND TRANFORM THE DEM #####
dem_aligned_fn = f.align_transform_pc(asp_dir, pc_deramped_fn, ref_dem_fn, roads_mask_fn, res, pc_align_dir, pc_transform_dir)


##### 5. FINAL RASTER ALIGNMENT #####
final_dem_fn, final_ddem_fn = f.raster_adjustments(ref_dem_adj_fn, dem_aligned_fn, ss_mask_fn, res, final_align_dir, vmin=-10, vmax=10)


##### 6. PLOT FINAL DEM AND DDEM #####
f.plot_dem_ddem(final_dem_fn, final_ddem_fn, roads_mask_fn, os.path.join(out_dir, 'final_dem_ddem.png'), vmin=-5, vmax=5)

## Apply terrain correction

In [ ]:
import xdem
import matplotlib.pyplot as plt
import geoutils as gu

print('Loading input files...')
dem = xdem.DEM(final_dem_fn)
ref_dem = xdem.DEM(ref_dem_adj_fn)
ref_dem = ref_dem.reproject(dem)
ss_mask = gu.Raster(ss_mask_fn, load_data=True)
ss_mask = ss_mask.reproject(dem)
ss_mask = (ss_mask==1)

# Test several terrain variables
terrain_vars = ['elevation', 'slope', 'aspect', 'maximum_curvature']
for terrain_var in terrain_vars:
    print(terrain_var)
    terrain_bias = xdem.coreg.TerrainBias(terrain_var).fit(ref_dem, dem, ss_mask)
    dem_corrected = terrain_bias.apply(dem)
    ddem_before = dem - ref_dem
    ddem_after = dem_corrected - ref_dem
    fig, ax = plt.subplots(1, 2, figsize=(12,6))
    ddem_before.plot(cmap='coolwarm_r', vmin=-5, vmax=5, ax=ax[0])
    ax[0].set_title('dDEM')
    ddem_after.plot(cmap='coolwarm_r', vmin=-5, vmax=5, ax=ax[1])
    ax[1].set_title('Corrected dDEM')
    plt.suptitle(terrain_var)
    plt.show()

## Try directional bias

In [ ]:
import numpy as np

for angle in np.linspace(0, 360, 5)[0:-1]:
    dir_bias = xdem.coreg.DirectionalBias(angle=angle).fit(ref_dem, dem, ss_mask)
    dem_corrected = dir_bias.apply(dem)
    ddem = dem_corrected - ref_dem
    fig, ax = plt.subplots(1, 1, figsize=(6,6))
    ddem.plot(cmap='coolwarm_r', vmin=-5, vmax=5, ax=ax)
    ax.set_title(angle)
    plt.show()

In [ ]:
deramp = xdem.coreg.Deramp().fit(ref_dem, dem, ss_mask)
dem_corrected = deramp.apply(dem)

ddem = dem_corrected - ref_dem

# roads_mask = gu.Raster(roads_mask_fn)
# roads_mask = roads_mask.reproject(dem)
# roads_mask = (roads_mask==1)
# ddem_roads = ddem[roads_mask]
# ddem_roads_med = np.nanmedian(ddem_roads.data)
# print(ddem_roads_med)

# ddem -= ddem_roads_med

ddem_ss = ddem[ss_mask]
ddem_ss_med = np.nanmedian(ddem_ss.data)
print(ddem_ss_med)

ddem.plot(cmap='coolwarm_r', vmin=-5, vmax=5)
plt.show()